In [1]:
import numpy as np
import simpy

from scipy.stats import expon, poisson, uniform

In [2]:
lam = 1

def arrivals(lam, S):
    while True:
        dt = expon.rvs(scale=1/lam) # exponential inter-arrival time distribution
        yield env.timeout(dt)
        S.append(env.now)

S = []
env = simpy.Environment() # sets up a simulation environment using SimPy
env.process(arrivals(lam, S))
env.run(until=10)
print(S)

[0.5865605280722542, 0.6695928285627948, 0.9188938518506995, 0.9642989092139909, 1.1333121392593832, 3.3134388068301632, 4.322387588697286, 4.919029294308295, 5.566098674857381, 5.65443383156199, 9.467165096468172]


In [3]:
env.timeout(expon.rvs(scale=1/lam))

<Timeout(0.020036523117305853) object at 0x2a4fee44850>

In [4]:
lam = 1

def arrivals(lam, S):
    while True:
        dt = expon.rvs(scale=1/lam)
        yield env.timeout(dt)
        env.process(service(server))

def service(server):
    rqt = server.request()
    yield rqt
    st = expon.rvs(scale=1/3)
    yield env.timeout(st)
    server.release(rqt)


env = simpy.Environment()
server = simpy.Resource(env=env, capacity=1)
env.process(arrivals(lam, server))
env.run(until=10)

In [17]:
lam = 1

def arrivals(lam, server):
    i = 0
    while True:
        dt = expon.rvs(scale=1/lam)
        if env.now + dt < 10:
            yield env.timeout(dt)
            print('person {} arrives at time {}'.format(i, env.now))
            env.process(service(server, i))
            i = i + 1
        else:
            break

def service(server, i):
    rqt = server.request()
    yield rqt
    print('person {} goes into service at time {}'.format(i, env.now))
    st = expon.rvs(scale=1/3)
    yield env.timeout(st)
    print('person {} releases the server at time {}'.format(i, env.now))
    server.release(rqt)


env = simpy.Environment()
server = simpy.Resource(env=env, capacity=1) # SimPy Resource object 
env.process(arrivals(lam, server))
env.run()

person 0 arrives at time 0.1881074291489169
[]
person 0 goes into service at time 0.1881074291489169
person 0 releases the server at time 0.21044614603287537
person 1 arrives at time 0.591386010296555
[]
person 1 goes into service at time 0.591386010296555
person 2 arrives at time 0.8321812451507794
person 1 releases the server at time 0.9945803280403132
[]
person 2 goes into service at time 0.9945803280403132
person 3 arrives at time 1.8331695510028214
person 4 arrives at time 2.3971487842126233
person 2 releases the server at time 2.6773668022350186
[<Request() object at 0x2a48f83a410>]
person 3 goes into service at time 2.6773668022350186
person 3 releases the server at time 2.9150211614292876
[]
person 4 goes into service at time 2.9150211614292876
person 4 releases the server at time 3.013715402848946
person 5 arrives at time 5.755423711451167
[]
person 5 goes into service at time 5.755423711451167
person 5 releases the server at time 5.7913286802822865
person 6 arrives at time 6.

In [14]:
server.users # list of the request that currently in resource, single server -> single request
server.count # number of people that currently in server, single server -> 0 or 1
server.queue # list of the request in order that currently in the queue of server
len(server.queue) # number of people in the queue
len(server.queue) + server.count # total number of people in system

0

In [13]:
server.count

0

implement a finite capacity single-server queue (arrivals accoring to homogeneous poisson pr)     
exponential service times     
capacity n     
if there are n people in system, arrivals are turned away     

In [7]:
lam = 1
n = 5

def arrivals(lam, server):
    i = 0
    while True:
        dt = expon.rvs(scale=1/lam)
        if env.now + dt < 10:
            yield env.timeout(dt)
            print('person {} arrives at time {}'.format(i, env.now))
            if server.count + len(server.queue) < n:
                env.process(service(server, i))
            else:
                print('person {} leaves at time {} because system full'.format(i, env.now))
            i = i + 1
        else:
            break

def service(server, i):
    rqt = server.request()
    yield rqt
    print('person {} goes into service at time {}'.format(i, env.now))
    st = expon.rvs(scale=1/3)
    yield env.timeout(st)
    print('person {} releases the server at time {}'.format(i, env.now))
    server.release(rqt)


env = simpy.Environment()
server = simpy.Resource(env=env, capacity=1)
env.process(arrivals(lam, server))
env.run()

person 0 arrives at time 0.2568127077836524
person 0 goes into service at time 0.2568127077836524
person 0 releases the server at time 0.6503177998560127
person 1 arrives at time 2.0885597581463156
person 1 goes into service at time 2.0885597581463156
person 1 releases the server at time 2.120899527730544
person 2 arrives at time 2.6620661787152136
person 2 goes into service at time 2.6620661787152136
person 3 arrives at time 2.731232745774811
person 2 releases the server at time 3.117991188815637
person 3 goes into service at time 3.117991188815637
person 3 releases the server at time 3.559559778586787
person 4 arrives at time 5.32186666706051
person 4 goes into service at time 5.32186666706051
person 4 releases the server at time 5.791534639014413
person 5 arrives at time 5.9400276559683896
person 5 goes into service at time 5.9400276559683896
person 5 releases the server at time 6.087748151875157
person 6 arrives at time 7.013326020108824
person 6 goes into service at time 7.0133260

In [8]:
# kitchen & bar

def arrivals(lam):
    while True:
        dt = expon.rvs(scale=1/lam)
        if env.now + dt < 10:
            yield env.timeout(dt)
            env.process(service(kitchen, food_time))
            env.process(service(bar, drink_time))
        else:
            break

def service(server, time):
    rqt = server.request()
    yield rqt
    st = expon.rvs(scale=1/3)
    yield env.timeout(st)
    server.release(rqt)
    time.append(env.now)

drink_time = []
food_time = []
env = simpy.Environment()
kitchen = simpy.Resource(env=env, capacity=1)
bar = simpy.Resource(env=env, capacity=1)
env.process(arrivals(lam))
env.run()

In [9]:
drink_time,food_time

([1.9886374794136252,
  2.278403676964766,
  2.433647028246644,
  2.551220015805925,
  2.564889522697002,
  5.115844673566181,
  6.096630794735699,
  6.368603485896093,
  6.536256015507288,
  7.25638659897314,
  8.525536067475882],
 [1.2765829985592116,
  1.993139049774007,
  2.3247581314159316,
  2.4305700246886373,
  2.6275552990008655,
  5.161686049324214,
  5.749927492409811,
  6.337050661553985,
  6.648527028255339,
  7.061254269007039,
  8.65978121233904])

In [10]:
def simulate():

    def arrivals(lam):
        while True:
            dt = expon.rvs(scale=1/lam)
            if env.now + dt < 10:
                yield env.timeout(dt)
                env.process(service(kitchen, food_time))
                env.process(service(bar, drink_time))
            else:
                break

    def service(server, time):
        rqt = server.request()
        yield rqt
        st = expon.rvs(scale=1/3)
        yield env.timeout(st)
        server.release(rqt)
        time.append(env.now)

    drink_time = []
    food_time = []
    env = simpy.Environment()
    kitchen = simpy.Resource(env=env, capacity=1)
    bar = simpy.Resource(env=env, capacity=1)
    env.process(arrivals(lam))
    env.run()

    drink_array = np.array(drink_time)
    food_array = np.array(food_time)

    max_diff = np.max(np.absolute(drink_array - food_array))

    return max_diff

def run_simulations(n):
    drink_times = [simulate() for i in range(n)]
    return drink_times




In [11]:
run_simulations(10)

[1.0889312779321125,
 0.6849447107225783,
 1.3819199449644604,
 0.9951920373595282,
 0.982697735758645,
 1.7025260754116414,
 0.4461848746348278,
 0.5392137487039905,
 1.1355182441934524,
 1.0880879028999964]